<a href="https://colab.research.google.com/github/samyeh0527/CNN_BiLSTM_withAttention/blob/master/Attention_with_CnnBiGRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/samyeh0527/CNN_BiLSTM_withAttention.git
%cd /content/CNN_BiLSTM_withAttention
!ls

Cloning into 'CNN_BiLSTM_withAttention'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 34 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (34/34), done.
/content/CNN_BiLSTM_withAttention
1210雙向注意力機制之深度學習模型應用於石油期貨價格之預測.pdf  Gold_lag4day.csv
Gold_lag13day.csv					      Gold_lag6day.csv
Gold_lag16day.csv					      Gold_lag8day.csv
Gold_lag17day.csv					      OilwithGold.csv
Gold_lag1day.csv					      README.md
Gold_lag2day.csv


In [ ]:
#多维NormalizeMult返回數據和最大最小值
import numpy 
import numpy as np

numpy.random.seed(7)
def NormalizeMult(data):
    #normalize 
    data = np.array(data)
    normalize = np.arange(2*data.shape[1],dtype='float64')

    normalize = normalize.reshape(data.shape[1],2)
    # print(normalize.shape)
    for i in range(0,data.shape[1]):
        list = data[:,i]
        listlow,listhigh =  np.percentile(list, [0, 100])
        # print(i)
        normalize[i,0] = listlow
        normalize[i,1] = listhigh
        delta = listhigh - listlow
        if delta != 0:
            for j in range(0,data.shape[0]):
                data[j,i]  =  (data[j,i] - listlow)/delta
    #np.save("./normalize.npy",normalize)
    return  data,normalize

#反正規
def FNormalizeMult(data,normalize):
    data = np.array(data)
    for i in  range(0,data.shape[1]):
        listlow =  normalize[i,0]
        listhigh = normalize[i,1]
        delta = listhigh - listlow
        if delta != 0:
            #第j行
            for j in range(0,data.shape[0]):
                data[j,i]  =  data[j,i]*delta + listlow

    return data
def attention_function(inputs, single_attention_vector=False):
   #inputs.shape = (batch_size, TimeSteps, Dims) 
    
    TimeSteps = K.int_shape(inputs)[1]
    input_dim = K.int_shape(inputs)[2]
    # print("TimeSteps",TimeSteps)
    # print("input_dim",input_dim)
    a = Permute((2, 1))(inputs)
    a = Dense(TimeSteps, activation='softmax')(a)
    if single_attention_vector:
        a = Lambda(lambda x: K.mean(x, axis=1))(a)
        a = RepeatVector(input_dim)(a)

    a_probs = Permute((2, 1))(a)
    # element * wise
    output_attention_mul = Multiply()([inputs, a_probs])
    return output_attention_mul

In [ ]:
from keras.layers import Input, Dense, LSTM, merge ,Conv1D,Dropout,Bidirectional,Multiply,Concatenate,BatchNormalization,GRU
from keras.models import Model
from sklearn.metrics import mean_squared_error
from keras.layers import merge
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.models import *
from keras.utils import plot_model
from keras import optimizers
import numpy
import  pandas as pd
import math
import datetime
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras import backend as K

starttime = datetime.datetime.now()
def NormalizeMult(data):
    #normalize 
    data = np.array(data)
    normalize = np.arange(2*data.shape[1],dtype='float64')

    normalize = normalize.reshape(data.shape[1],2)
    # print(normalize.shape)
    for i in range(0,data.shape[1]):
        list = data[:,i]
        listlow,listhigh =  np.percentile(list, [0, 100])
        # print(i)
        normalize[i,0] = listlow
        normalize[i,1] = listhigh
        delta = listhigh - listlow
        if delta != 0:
            for j in range(0,data.shape[0]):
                data[j,i]  =  (data[j,i] - listlow)/delta
    #np.save("./normalize.npy",normalize)
    return  data,normalize

#反正規
def FNormalizeMult(data,normalize):
    data = np.array(data)
    for i in  range(0,data.shape[1]):
        listlow =  normalize[i,0]
        listhigh = normalize[i,1]
        delta = listhigh - listlow
        if delta != 0:
            #第j行
            for j in range(0,data.shape[0]):
                data[j,i]  =  data[j,i]*delta + listlow

    return data
def attention_function(inputs, single_attention_vector=False):
   #inputs.shape = (batch_size, TimeSteps, Dims) 
    
    TimeSteps = K.int_shape(inputs)[1]
    input_dim = K.int_shape(inputs)[2]
    a = Permute((2, 1))(inputs)
    a = Dense(TimeSteps, activation='softmax')(a)
    if single_attention_vector:
        a = Lambda(lambda x: K.mean(x, axis=1))(a)
        a = RepeatVector(input_dim)(a)

    a_probs = Permute((2, 1))(a)
    # element * wise
    output_attention_mul = Multiply()([inputs, a_probs])
    return output_attention_mul

# convert an array of values into a dataset matrix
def create_dataset2(dataset, look_back):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back),:]
        dataX.append(a)
        dataY.append(dataset[i + look_back,:])
    TrainX = numpy.array(dataX)
    Train_Y = numpy.array(dataY)

    return TrainX, Train_Y 
# fix random seed for reproducibility
numpy.random.seed(7)
# load the dataset
dataframe = read_csv('/content/CNN_BiLSTM_withAttention/OilwithGold.csv')
dataframe = dataframe.drop(['Date'], axis = 1)
dataset = dataframe.values
dataset = dataset.astype('float32')
testVaild=dataset
trainVaild=dataset




# split into train and test sets
train_size = int(len(dataset) * 0.7)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
testVaild = testVaild[train_size:len(dataset),:]
trainVaild = trainVaild[:train_size,:]
print(train.shape)
print(test.shape)
# normalize the dataset
dataset,da_normalize = NormalizeMult(dataset)
train,tr_normalize = NormalizeMult(train)
test,te_normalize = NormalizeMult(test)


look_back = 2
TimeSteps=look_back
Dims=2
trainX, trY = create_dataset2(train, look_back)
testX, teY = create_dataset2(test, look_back)
print(trainX.shape,trY.shape)
print(testX.shape,teY.shape)
trainY =trY[:,0]
testY =teY[:,0]
print(trainY.shape)
print(testY.shape)


def attention_model():
    inputs = Input(shape=(TimeSteps, Dims))
    x = Conv1D(filters = 128, kernel_size = 1, activation = 'relu')(inputs)  
    BiGRU_out = Bidirectional(GRU(64, return_sequences=True,activation="relu"))(x)
    Batch_Normalization = BatchNormalization()(BiGRU_out)
    Drop_out = Dropout(0.1)(Batch_Normalization)
    attention = attention_function(Drop_out)
    Batch_Normalization = BatchNormalization()(attention)
    Drop_out = Dropout(0.1)(Batch_Normalization)
    Flatten_ = Flatten()(Drop_out)
    output=Dropout(0.1)(Flatten_)
    output = Dense(1, activation='sigmoid')(output)
    model = Model(inputs=[inputs], outputs=output)
    return model

m = attention_model()
m.summary()
m.compile(loss='mean_squared_error', optimizer='adam')
m.fit(trainX, trainY, epochs=30, batch_size=64, verbose=0,validation_data=(testX, testY))
# make predictions
trainPredict = m.predict(trainX)
testPredict = m.predict(testX)
#FNormalize
trainPredict_FNormalizeMult= FNormalizeMult(trainPredict,tr_normalize)
testPredict_FNormalizeMult= FNormalizeMult(testPredict,te_normalize)

#calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict))

print('*  Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict))
print('*  Test Score: %.2f RMSE' % (testScore))
teV=testVaild[:-(look_back+1),0] 
trV=trainVaild[:-(look_back+1),0] 
print("---")
print("*  look_back = ",look_back)
trainScore = math.sqrt(mean_squared_error(trV, trainPredict_FNormalizeMult))
print('*  FNormalizeMult Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(teV, testPredict_FNormalizeMult))
print('*  FNormalizeMult Test Score: %.2f RMSE' % (testScore))
# shift test predictions for plotting
                                       
plt.plot(teV)       
plt.plot(testPredict_FNormalizeMult,'r')
plt.show()
plot_model(m, show_shapes=True, to_file='AttentionCnnBiGRU.png')
plt.plot(trV)
plt.plot(trainPredict_FNormalizeMult,'r')
plt.show()

ImportError: ignored

In [ ]:
import csv

# 開啟輸出的 CSV 檔案
with open('testPredict_FNormalizeMult.csv', 'w', newline='') as csvfile:
  # 建立 CSV 檔寫入器
  writer = csv.writer(csvfile)
  writer.writerows(testPredict_FNormalizeMult)

with open('trainPredict_FNormalizeMult.csv', 'w', newline='') as csvfile:
  # 建立 CSV 檔寫入器
  writer = csv.writer(csvfile)
  writer.writerows(trainPredict_FNormalizeMult)


96 train 4 test Att with CNNBiGRU
---
*  look_back =  1
*  Train Score: 0.04 RMSE
*  Test Score: 0.08 RMSE
*  FNormalizeMult Test Score: 3.60 RMSE
---
* loob_back = 2
* Train Score: 0.02 RMSE
* Test Score: 0.07 RMSE
* FNormalizeMult Test Score: 3.79 RMSE
---
*  look_back =  3
*  Train Score: 0.01 RMSE
*  Test Score: 0.07 RMSE
*  FNormalizeMult Test Score: 3.23 RMSE
---
*  look_back =  4
*  Train Score: 0.03 RMSE
*  Test Score: 0.09 RMSE
*  FNormalizeMult Test Score: 4.77 RMSE
---
*  look_back =  5
*  Train Score: 0.03 RMSE
*  Test Score: 0.10 RMSE
*  FNormalizeMult Test Score: 5.24 RMSE
---
*  look_back =  6
*  Train Score: 0.02 RMSE
*  Test Score: 0.09 RMSE
*  FNormalizeMult Test Score: 6.33 RMSE
---
*  look_back =  7
*  Train Score: 0.02 RMSE
*  Test Score: 0.06 RMSE
*  FNormalizeMult Test Score: 5.77 RMSE
---
*  look_back =  8
*  Train Score: 0.02 RMSE
*  Test Score: 0.10 RMSE
*  FNormalizeMult Test Score: 6.88 RMSE
---
*  look_back =  9
*  Train Score: 0.03 RMSE
*  Test Score: 0.08 RMSE
*  FNormalizeMult Test Score: 7.42 RMSE
---
*  look_back =  10
*  Train Score: 0.03 RMSE
*  Test Score: 0.08 RMSE
*  FNormalizeMult Test Score: 7.85 RMSE
---

*  look_back =  11
*  FNormalizeMult Train Score: 5.05 RMSE
*  FNormalizeMult Test Score: 8.88 RMSE
---
*  look_back =  12
*  FNormalizeMult Train Score: 4.38 RMSE
*  FNormalizeMult Test Score: 9.44 RMSE
---
*  look_back =  13

*  FNormalizeMult Test Score: 10.07RMSE
---
*  look_back =  14
*  FNormalizeMult Train Score: 4.59 RMSE
*  FNormalizeMult Test Score: 10.77 RMSE
---
*  look_back =  15
*  FNormalizeMult Train Score: 4.63 RMSE
*  FNormalizeMult Test Score: 11.00 RMSE
---
---
*  look_back =  16
*  FNormalizeMult Train Score: 5.08 RMSE
*  FNormalizeMult Test Score: 11.38 RMSE
---
*  look_back =  17
*  FNormalizeMult Train Score: 7.89 RMSE
*  FNormalizeMult Test Score: 11.93 RMSE
---
*  look_back =  18
*  FNormalizeMult Train Score: 5.43 RMSE
*  FNormalizeMult Test Score: 12.22 RMSE
---
*  look_back =  19
*  FNormalizeMult Train Score: 6.17 RMSE
*  FNormalizeMult Test Score: 12.32 RMSE
---
*  look_back =  19
*  FNormalizeMult Train Score: 6.17 RMSE
*  FNormalizeMult Test Score: 12.32 RMSE
---
*  look_back =  20
*  FNormalizeMult Train Score: 6.24 RMSE
*  FNormalizeMult Test Score: 12.95 RMSE